In [209]:
# librerias
from __future__ import print_function
import mysql.connector
import pandas as pd
#import glob
#import re

In [210]:
# Me conecto a la base de datos
cnx = mysql.connector.connect(user='root', password="agustin", database='mycoDB')
print(cnx)

In [211]:
# Borro los datos de las tablas

try:
    cursor = cnx.cursor()

    cursor.execute("DELETE FROM motifs")
    cursor.execute("DELETE FROM redRTLocus")
    cursor.execute("DELETE FROM redRTOperons")
    cursor.execute("DELETE FROM locus")
    cursor.execute("DELETE FROM operons")
    cnx.commit()
    print(cnx.commit())
    
finally:
    cursor.close()

None


In [212]:
# Cargo la tabla Operones

# Leo el archivo donde esta en mfinderID de cada operon
operonID_mfinderID=pd.read_csv("2_redT_conOperones_fullNames.txt",sep='\t',header=None)
# Creo un indice donce la llave es el operon y el value el mfinderID
operonID_mfinderID_Index={}
# Lleno el indice
for line in range(len(operonID_mfinderID)):
    operonID_mfinderID_Index[operonID_mfinderID.iloc[line][1]]=int(operonID_mfinderID.iloc[line][0])

# Leo el archivo con todos los operones
operon_file=pd.read_csv("1_operones_newID.opr",sep='\t')
operons=(operon_file["OperonID"])

# Inicio el cursor
cursor = cnx.cursor()

try:  
    for line in list(range(len(operons))): 
        # Inserto los operones y busco en eindice de mfinderID si tienen Id de motivos, sino inserto iun NULL
        # Inserto solo las no repetidas
        add = ("INSERT INTO operons (operonID, mfinderID) VALUES (%(a)s, %(b)s)")
        data = {"a":str(operons[line]), "b": operonID_mfinderID_Index.get(str(operons[line]),None)}
        cursor.execute(add,data)
        cnx.commit()
    for key in operonID_mfinderID_Index:
        # Inserto los operones que son denominados como locus, ya que no estan en ningun operon.
        if key.startswith("rv"):
            add = ("INSERT INTO operons (operonID, mfinderID) VALUES (%(a)s, %(b)s)")
            data = {"a":key, "b": operonID_mfinderID_Index[key]}
            cursor.execute(add,data)
            cnx.commit()
            
finally:  
    cursor.close()

print("OK")    

{'a': '1132034', 'b': 1}
{'a': '1132034', 'b': 1}
{'a': '1132034', 'b': 1}
{'a': '1132035', 'b': 2}
{'a': '1132035', 'b': 2}
{'a': '1132036', 'b': None}
{'a': '1132036', 'b': None}
{'a': '1132037', 'b': 3}
{'a': '1132037', 'b': 3}
{'a': '1132038', 'b': 4}
{'a': '1132038', 'b': 4}
{'a': '1132038', 'b': 4}
{'a': '1132038', 'b': 4}
{'a': '1132038', 'b': 4}
{'a': '1132038', 'b': 4}
{'a': '1132039', 'b': 5}
{'a': '1132039', 'b': 5}
{'a': '1132040', 'b': None}
{'a': '1132040', 'b': None}
{'a': '1132041', 'b': None}
{'a': '1132041', 'b': None}
{'a': '1132042', 'b': 6}
{'a': '1132042', 'b': 6}
{'a': '1132042', 'b': 6}
{'a': '1132042', 'b': 6}
{'a': '1132043', 'b': None}
{'a': '1132043', 'b': None}
{'a': '1132044', 'b': None}
{'a': '1132044', 'b': None}
{'a': '1132045', 'b': 7}
{'a': '1132045', 'b': 7}
{'a': '1132046', 'b': None}
{'a': '1132046', 'b': None}
{'a': '1132046', 'b': None}
{'a': '1132047', 'b': 8}
{'a': '1132047', 'b': 8}
{'a': '1132047', 'b': 8}
{'a': '1132047', 'b': 8}
{'a': '1132

{'a': '1132143', 'b': 64}
{'a': '1132143', 'b': 64}
{'a': '1132144', 'b': None}
{'a': '1132144', 'b': None}
{'a': '1132144', 'b': None}
{'a': '1132145', 'b': None}
{'a': '1132145', 'b': None}
{'a': '1132145', 'b': None}
{'a': '1132145', 'b': None}
{'a': '1132145', 'b': None}
{'a': '1132146', 'b': None}
{'a': '1132146', 'b': None}
{'a': '1132146', 'b': None}
{'a': '1132147', 'b': None}
{'a': '1132147', 'b': None}
{'a': '1132148', 'b': 65}
{'a': '1132148', 'b': 65}
{'a': '1132148', 'b': 65}
{'a': '1132149', 'b': None}
{'a': '1132149', 'b': None}
{'a': '1132149', 'b': None}
{'a': '1132149', 'b': None}
{'a': '1132149', 'b': None}
{'a': '1132149', 'b': None}
{'a': '1132150', 'b': 66}
{'a': '1132150', 'b': 66}
{'a': '1132151', 'b': 67}
{'a': '1132151', 'b': 67}
{'a': '1132151', 'b': 67}
{'a': '1132151', 'b': 67}
{'a': '1132151', 'b': 67}
{'a': '1132151', 'b': 67}
{'a': '1132151', 'b': 67}
{'a': '1132152', 'b': None}
{'a': '1132152', 'b': None}
{'a': '1132153', 'b': None}
{'a': '1132153', 'b'

{'a': '1132252', 'b': 122}
{'a': '1132252', 'b': 122}
{'a': '1132253', 'b': 123}
{'a': '1132253', 'b': 123}
{'a': '1132254', 'b': 124}
{'a': '1132254', 'b': 124}
{'a': '1132254', 'b': 124}
{'a': '1132255', 'b': 125}
{'a': '1132255', 'b': 125}
{'a': '1132256', 'b': 126}
{'a': '1132256', 'b': 126}
{'a': '1132256', 'b': 126}
{'a': '1132256', 'b': 126}
{'a': '1132257', 'b': 127}
{'a': '1132257', 'b': 127}
{'a': '1132258', 'b': None}
{'a': '1132258', 'b': None}
{'a': '1132259', 'b': 128}
{'a': '1132259', 'b': 128}
{'a': '1132260', 'b': 129}
{'a': '1132260', 'b': 129}
{'a': '1132261', 'b': 130}
{'a': '1132261', 'b': 130}
{'a': '1132262', 'b': None}
{'a': '1132262', 'b': None}
{'a': '1132263', 'b': None}
{'a': '1132263', 'b': None}
{'a': '1132264', 'b': 131}
{'a': '1132264', 'b': 131}
{'a': '1132265', 'b': 132}
{'a': '1132265', 'b': 132}
{'a': '1132265', 'b': 132}
{'a': '1132266', 'b': 133}
{'a': '1132266', 'b': 133}
{'a': '1132266', 'b': 133}
{'a': '1132266', 'b': 133}
{'a': '1132266', 'b': 

{'a': '1132362', 'b': 184}
{'a': '1132363', 'b': None}
{'a': '1132363', 'b': None}
{'a': '1132363', 'b': None}
{'a': '1132364', 'b': 185}
{'a': '1132364', 'b': 185}
{'a': '1132365', 'b': 186}
{'a': '1132365', 'b': 186}
{'a': '1132366', 'b': 187}
{'a': '1132366', 'b': 187}
{'a': '1132367', 'b': 188}
{'a': '1132367', 'b': 188}
{'a': '1132367', 'b': 188}
{'a': '1132367', 'b': 188}
{'a': '1132368', 'b': 189}
{'a': '1132368', 'b': 189}
{'a': '1132369', 'b': None}
{'a': '1132369', 'b': None}
{'a': '1132370', 'b': 190}
{'a': '1132370', 'b': 190}
{'a': '1132370', 'b': 190}
{'a': '1132370', 'b': 190}
{'a': '1132371', 'b': 191}
{'a': '1132371', 'b': 191}
{'a': '1132371', 'b': 191}
{'a': '1132371', 'b': 191}
{'a': '1132372', 'b': 192}
{'a': '1132372', 'b': 192}
{'a': '1132372', 'b': 192}
{'a': '1132373', 'b': None}
{'a': '1132373', 'b': None}
{'a': '1132374', 'b': 193}
{'a': '1132374', 'b': 193}
{'a': '1132374', 'b': 193}
{'a': '1132375', 'b': None}
{'a': '1132375', 'b': None}
{'a': '1132376', 'b

{'a': '1132461', 'b': 236}
{'a': '1132462', 'b': None}
{'a': '1132462', 'b': None}
{'a': '1132463', 'b': 237}
{'a': '1132463', 'b': 237}
{'a': '1132463', 'b': 237}
{'a': '1132463', 'b': 237}
{'a': '1132464', 'b': 238}
{'a': '1132464', 'b': 238}
{'a': '1132465', 'b': 239}
{'a': '1132465', 'b': 239}
{'a': '1132465', 'b': 239}
{'a': '1132466', 'b': None}
{'a': '1132466', 'b': None}
{'a': '1132467', 'b': None}
{'a': '1132467', 'b': None}
{'a': '1132468', 'b': None}
{'a': '1132468', 'b': None}
{'a': '1132469', 'b': 240}
{'a': '1132469', 'b': 240}
{'a': '1132469', 'b': 240}
{'a': '1132469', 'b': 240}
{'a': '1132470', 'b': 241}
{'a': '1132470', 'b': 241}
{'a': '1132470', 'b': 241}
{'a': '1132470', 'b': 241}
{'a': '1132470', 'b': 241}
{'a': '1132470', 'b': 241}
{'a': '1132470', 'b': 241}
{'a': '1132471', 'b': None}
{'a': '1132471', 'b': None}
{'a': '1132472', 'b': 242}
{'a': '1132472', 'b': 242}
{'a': '1132472', 'b': 242}
{'a': '1132472', 'b': 242}
{'a': '1132473', 'b': None}
{'a': '1132473', 

...

{'a': '3873107', 'b': None}
{'a': '3873108', 'b': 514}
{'a': '3873109', 'b': None}
{'a': '3873110', 'b': None}
{'a': '3873111', 'b': None}
{'a': '3873112', 'b': 515}
{'a': '3873113', 'b': None}
{'a': '3873114', 'b': None}
{'a': '3873115', 'b': 516}
{'a': '3873116', 'b': 517}
{'a': '3873117', 'b': None}
{'a': '3873118', 'b': None}
{'a': '3873119', 'b': None}
{'a': '3873120', 'b': None}
{'a': '3873121', 'b': None}
{'a': '3873122', 'b': None}
{'a': '3873123', 'b': 518}
{'a': '3873124', 'b': 519}
{'a': '3873125', 'b': 520}
{'a': '3873126', 'b': None}
{'a': '3873127', 'b': None}
{'a': '3873128', 'b': None}
{'a': '3873129', 'b': 521}
{'a': '3873130', 'b': 522}
{'a': '3873131', 'b': None}
{'a': '3873132', 'b': None}
{'a': '3873133', 'b': None}
{'a': '3873134', 'b': None}
{'a': '3873135', 'b': 523}
{'a': '3873136', 'b': None}
{'a': '3873137', 'b': None}
{'a': '3873138', 'b': 524}
{'a': '3873139', 'b': None}
{'a': '3873140', 'b': None}
{'a': '3873141', 'b': 525}
{'a': '3873142', 'b': 526}
{'a':

{'a': '3873404', 'b': None}
{'a': '3873405', 'b': 602}
{'a': '3873406', 'b': 603}
{'a': '3873407', 'b': None}
{'a': '3873408', 'b': None}
{'a': '3873409', 'b': None}
{'a': '3873410', 'b': None}
{'a': '3873411', 'b': 604}
{'a': '3873412', 'b': None}
{'a': '3873413', 'b': 605}
{'a': '3873414', 'b': 606}
{'a': '3873415', 'b': 607}
{'a': '3873416', 'b': 608}
{'a': '3873417', 'b': 609}
{'a': '3873418', 'b': None}
{'a': '3873419', 'b': 610}
{'a': '3873420', 'b': None}
{'a': '3873421', 'b': 611}
{'a': '3873422', 'b': None}
{'a': '3873423', 'b': 612}
{'a': '3873424', 'b': None}
{'a': '3873425', 'b': None}
{'a': '3873426', 'b': 613}
{'a': '3873427', 'b': None}
{'a': '3873428', 'b': 614}
{'a': '3873429', 'b': None}
{'a': '3873430', 'b': None}
{'a': '3873431', 'b': None}
{'a': '3873432', 'b': 615}
{'a': '3873433', 'b': 616}
{'a': '3873434', 'b': None}
{'a': '3873435', 'b': None}
{'a': '3873436', 'b': None}
{'a': '3873437', 'b': 617}
{'a': '3873438', 'b': 618}
{'a': '3873439', 'b': None}
{'a': '38

{'a': '3873702', 'b': 707}
{'a': '3873703', 'b': None}
{'a': '3873704', 'b': None}
{'a': '3873705', 'b': None}
{'a': '3873706', 'b': None}
{'a': '3873707', 'b': 708}
{'a': '3873708', 'b': None}
{'a': '3873709', 'b': None}
{'a': '3873710', 'b': None}
{'a': '3873711', 'b': None}
{'a': '3873712', 'b': None}
{'a': '3873713', 'b': None}
{'a': '3873714', 'b': None}
{'a': '3873715', 'b': None}
{'a': '3873716', 'b': None}
{'a': '3873717', 'b': 709}
{'a': '3873718', 'b': 710}
{'a': '3873719', 'b': 711}
{'a': '3873720', 'b': 712}
{'a': '3873721', 'b': None}
{'a': '3873722', 'b': 713}
{'a': '3873723', 'b': 714}
{'a': '3873724', 'b': None}
{'a': '3873725', 'b': 715}
{'a': '3873726', 'b': None}
{'a': '3873727', 'b': None}
{'a': '3873728', 'b': None}
{'a': '3873729', 'b': None}
{'a': '3873730', 'b': None}
{'a': '3873731', 'b': 716}
{'a': '3873732', 'b': None}
{'a': '3873733', 'b': None}
{'a': '3873734', 'b': None}
{'a': '3873735', 'b': None}
{'a': '3873736', 'b': None}
{'a': '3873737', 'b': None}
{'

{'a': '3874000', 'b': None}
{'a': '3874001', 'b': None}
{'a': '3874002', 'b': 805}
{'a': '3874003', 'b': None}
{'a': '3874004', 'b': None}
{'a': '3874005', 'b': None}
{'a': '3874006', 'b': None}
{'a': '3874007', 'b': None}
{'a': '3874008', 'b': None}
{'a': '3874009', 'b': None}
{'a': '3874010', 'b': None}
{'a': '3874011', 'b': None}
{'a': '3874012', 'b': None}
{'a': '3874013', 'b': 806}
{'a': '3874014', 'b': 807}
{'a': '3874015', 'b': None}
{'a': '3874016', 'b': None}
{'a': '3874017', 'b': None}
{'a': '3874018', 'b': None}
{'a': '3874019', 'b': None}
{'a': '3874020', 'b': None}
{'a': '3874021', 'b': None}
{'a': '3874022', 'b': None}
{'a': '3874023', 'b': None}
{'a': '3874024', 'b': 808}
{'a': '3874025', 'b': None}
{'a': '3874026', 'b': 809}
{'a': '3874027', 'b': None}
{'a': '3874028', 'b': 810}
{'a': '3874029', 'b': 811}
{'a': '3874030', 'b': None}
{'a': '3874031', 'b': None}
{'a': '3874032', 'b': None}
{'a': '3874033', 'b': None}
{'a': '3874034', 'b': None}
{'a': '3874035', 'b': None}

{'a': '3874297', 'b': 890}
{'a': '3874298', 'b': 891}
{'a': '3874299', 'b': None}
{'a': '3874300', 'b': 892}
{'a': '3874301', 'b': None}
{'a': '3874302', 'b': 893}
{'a': '3874303', 'b': None}
{'a': '3874304', 'b': None}
{'a': '3874305', 'b': None}
{'a': '3874306', 'b': None}
{'a': '3874307', 'b': 894}
{'a': '3874308', 'b': 895}
{'a': '3874309', 'b': None}
{'a': '3874310', 'b': None}
{'a': '3874311', 'b': 896}
{'a': '3874312', 'b': 897}
{'a': '3874313', 'b': 898}
{'a': '3874314', 'b': 899}
{'a': '3874315', 'b': None}
{'a': '3874316', 'b': None}
{'a': '3874317', 'b': None}
{'a': '3874318', 'b': None}
{'a': '3874319', 'b': None}
{'a': '3874320', 'b': 900}
{'a': '3874321', 'b': None}
{'a': '3874322', 'b': None}
{'a': '3874323', 'b': None}
{'a': '3874324', 'b': 901}
{'a': '3874325', 'b': None}
{'a': '3874326', 'b': None}
{'a': '3874327', 'b': 902}
{'a': '3874328', 'b': 903}
{'a': '3874329', 'b': 904}
{'a': '3874330', 'b': None}
{'a': '3874331', 'b': 905}
{'a': '3874332', 'b': 906}
{'a': '38

In [213]:
# Cargo la tabla locus

# Leo el archivo con todos los locus
loci_file=open("listaDelocush37rv.txt","r")
# Leo el archivo con todos los operones
operones_file=pd.read_csv('1_operones_newID.opr',sep='\t')

# Parseo operones y creo un indice de operones-locus
index_loci_operon={}
operon= operones_file["OperonID"]
locusID= operones_file["Synonym"]
for fila in list(range(len(operon))):
    index_loci_operon[locusID[fila]]=int(operon[fila])
     
# Inicio el cursor
cursor = cnx.cursor()
i=0
try:
    for locus in loci_file:
        i=i+1
        locus=locus.rstrip()
        add = ("INSERT INTO locus (locusID, operonID) VALUES (%(a)s, %(b)s)")
        data = {"a": locus, "b": index_loci_operon.get(str(locus),None)}
        #print(data)
        cursor.execute(add,data)       
        cnx.commit()
finally:  
    cursor.close()
    loci_file.close()
    
print("OK")

OK


In [214]:
# Cargo la tabla redRTLocus

# Leo el archivo con la red de Locus
redRT_file=pd.read_csv('1_myco_h37rv_RT_locus.txt',sep='\t',header=None)

# Inicio el cursor
cursor = cnx.cursor()

try:
    for line in range(len(redRT_file)):
        nodos=redRT_file.iloc[line][1].split()
        regulador=nodos[0]
        regulado=nodos[1]        
        add = ("INSERT INTO redRTLocus (regulador, regulado) VALUES (%(a)s, %(b)s)")
        data = {"a": regulador, "b": regulado}
        #print(data)
        cursor.execute(add,data)       
        cnx.commit()
finally:  
    cursor.close()
print("OK")

OK


In [215]:
# Cargo la tabla redRTOperons, con mfinderIDs

# Leo el archivo con la red de Operones pero con mfinderID
redRT_mfinderID_file=pd.read_csv('2_redT_conOperones_net_number.txt',sep='\s+',header=None)

# Inicio el cursor
cursor = cnx.cursor()

try:
    for line in range(len(redRT_mfinderID_file)):
        regulador=int(redRT_mfinderID_file.iloc[line][0])
        regulado=int(redRT_mfinderID_file.iloc[line][1])
        add = ("INSERT INTO redRTOperons (A, B) VALUES (%(a)s, %(b)s)")
        data = {"a": regulador, "b": regulado}
        cursor.execute(add,data)       
        cnx.commit()
finally:  
    cursor.close()
print("OK")

{'a': 489, 'b': 1}
{'a': 489, 'b': 413}
{'a': 489, 'b': 417}
{'a': 507, 'b': 246}
{'a': 507, 'b': 306}
{'a': 507, 'b': 486}
{'a': 28, 'b': 73}
{'a': 28, 'b': 210}
{'a': 41, 'b': 527}
{'a': 41, 'b': 681}
{'a': 47, 'b': 526}
{'a': 47, 'b': 34}
{'a': 47, 'b': 541}
{'a': 47, 'b': 542}
{'a': 64, 'b': 138}
{'a': 64, 'b': 227}
{'a': 64, 'b': 409}
{'a': 64, 'b': 893}
{'a': 553, 'b': 11}
{'a': 553, 'b': 552}
{'a': 553, 'b': 581}
{'a': 553, 'b': 111}
{'a': 553, 'b': 124}
{'a': 553, 'b': 126}
{'a': 553, 'b': 127}
{'a': 553, 'b': 135}
{'a': 553, 'b': 169}
{'a': 553, 'b': 225}
{'a': 553, 'b': 301}
{'a': 554, 'b': 512}
{'a': 554, 'b': 22}
{'a': 554, 'b': 72}
{'a': 75, 'b': 115}
{'a': 75, 'b': 146}
{'a': 100, 'b': 153}
{'a': 100, 'b': 223}
{'a': 100, 'b': 356}
{'a': 586, 'b': 67}
{'a': 586, 'b': 224}
{'a': 586, 'b': 914}
{'a': 603, 'b': 132}
{'a': 134, 'b': 649}
{'a': 134, 'b': 825}
{'a': 140, 'b': 141}
{'a': 142, 'b': 619}
{'a': 649, 'b': 526}
{'a': 649, 'b': 34}
{'a': 649, 'b': 59}
{'a': 649, 'b': 

{'a': 932, 'b': 230}
{'a': 932, 'b': 701}
{'a': 932, 'b': 704}
{'a': 932, 'b': 705}
{'a': 932, 'b': 709}
{'a': 932, 'b': 731}
{'a': 932, 'b': 732}
{'a': 932, 'b': 256}
{'a': 932, 'b': 269}
{'a': 932, 'b': 270}
{'a': 932, 'b': 273}
{'a': 932, 'b': 753}
{'a': 932, 'b': 286}
{'a': 932, 'b': 768}
{'a': 932, 'b': 287}
{'a': 932, 'b': 771}
{'a': 932, 'b': 774}
{'a': 932, 'b': 775}
{'a': 932, 'b': 778}
{'a': 932, 'b': 780}
{'a': 932, 'b': 300}
{'a': 932, 'b': 787}
{'a': 932, 'b': 789}
{'a': 932, 'b': 790}
{'a': 932, 'b': 791}
{'a': 932, 'b': 792}
{'a': 932, 'b': 796}
{'a': 932, 'b': 797}
{'a': 932, 'b': 309}
{'a': 932, 'b': 800}
{'a': 932, 'b': 811}
{'a': 932, 'b': 330}
{'a': 932, 'b': 822}
{'a': 932, 'b': 823}
{'a': 932, 'b': 353}
{'a': 932, 'b': 845}
{'a': 932, 'b': 373}
{'a': 932, 'b': 852}
{'a': 932, 'b': 855}
{'a': 932, 'b': 377}
{'a': 932, 'b': 378}
{'a': 932, 'b': 871}
{'a': 932, 'b': 397}
{'a': 932, 'b': 878}
{'a': 932, 'b': 403}
{'a': 932, 'b': 405}
{'a': 932, 'b': 406}
{'a': 932, 'b

{'a': 46, 'b': 894}
{'a': 46, 'b': 897}
{'a': 46, 'b': 898}
{'a': 46, 'b': 900}
{'a': 46, 'b': 449}
{'a': 46, 'b': 456}
{'a': 46, 'b': 457}
{'a': 46, 'b': 938}
{'a': 46, 'b': 464}
{'a': 46, 'b': 466}
{'a': 46, 'b': 469}
{'a': 46, 'b': 944}
{'a': 46, 'b': 13}
{'a': 46, 'b': 509}
{'a': 46, 'b': 516}
{'a': 46, 'b': 520}
{'a': 46, 'b': 31}
{'a': 46, 'b': 530}
{'a': 46, 'b': 45}
{'a': 46, 'b': 71}
{'a': 46, 'b': 72}
{'a': 46, 'b': 88}
{'a': 46, 'b': 579}
{'a': 46, 'b': 117}
{'a': 46, 'b': 603}
{'a': 46, 'b': 132}
{'a': 46, 'b': 173}
{'a': 46, 'b': 177}
{'a': 46, 'b': 196}
{'a': 46, 'b': 207}
{'a': 46, 'b': 232}
{'a': 46, 'b': 233}
{'a': 46, 'b': 700}
{'a': 46, 'b': 712}
{'a': 46, 'b': 240}
{'a': 46, 'b': 720}
{'a': 46, 'b': 252}
{'a': 46, 'b': 736}
{'a': 46, 'b': 737}
{'a': 46, 'b': 743}
{'a': 46, 'b': 966}
{'a': 46, 'b': 282}
{'a': 46, 'b': 773}
{'a': 46, 'b': 313}
{'a': 46, 'b': 806}
{'a': 46, 'b': 813}
{'a': 46, 'b': 324}
{'a': 46, 'b': 814}
{'a': 46, 'b': 815}
{'a': 46, 'b': 832}
{'a': 

{'a': 400, 'b': 896}
{'a': 400, 'b': 543}
{'a': 400, 'b': 168}
{'a': 400, 'b': 164}
{'a': 400, 'b': 22}
{'a': 400, 'b': 26}
{'a': 400, 'b': 65}
{'a': 400, 'b': 305}
{'a': 400, 'b': 489}
{'a': 400, 'b': 658}
{'a': 400, 'b': 1}
{'a': 400, 'b': 24}
{'a': 400, 'b': 684}
{'a': 400, 'b': 362}
{'a': 400, 'b': 677}
{'a': 400, 'b': 637}
{'a': 400, 'b': 237}
{'a': 400, 'b': 383}
{'a': 400, 'b': 675}
{'a': 400, 'b': 639}
{'a': 400, 'b': 148}
{'a': 400, 'b': 118}
{'a': 400, 'b': 496}
{'a': 400, 'b': 877}
{'a': 400, 'b': 388}
{'a': 400, 'b': 25}
{'a': 400, 'b': 222}
{'a': 400, 'b': 508}
{'a': 400, 'b': 116}
{'a': 400, 'b': 763}
{'a': 400, 'b': 200}
{'a': 400, 'b': 281}
{'a': 400, 'b': 88}
{'a': 400, 'b': 660}
{'a': 400, 'b': 122}
{'a': 400, 'b': 169}
{'a': 400, 'b': 402}
{'a': 400, 'b': 178}
{'a': 400, 'b': 172}
{'a': 400, 'b': 83}
{'a': 400, 'b': 622}
{'a': 400, 'b': 613}
{'a': 400, 'b': 157}
{'a': 400, 'b': 52}
{'a': 400, 'b': 726}
{'a': 400, 'b': 21}
{'a': 400, 'b': 694}
{'a': 400, 'b': 632}
{'a

{'a': 553, 'b': 817}
{'a': 553, 'b': 770}
{'a': 553, 'b': 873}
{'a': 553, 'b': 94}
{'a': 553, 'b': 9}
{'a': 553, 'b': 673}
{'a': 553, 'b': 329}
{'a': 553, 'b': 640}
{'a': 553, 'b': 641}
{'a': 553, 'b': 642}
{'a': 553, 'b': 795}
{'a': 553, 'b': 719}
{'a': 46, 'b': 571}
{'a': 46, 'b': 96}
{'a': 46, 'b': 211}
{'a': 46, 'b': 840}
{'a': 46, 'b': 895}
{'a': 46, 'b': 487}
{'a': 46, 'b': 850}
{'a': 46, 'b': 9}
{'a': 46, 'b': 20}
{'a': 46, 'b': 962}
{'a': 46, 'b': 639}
{'a': 46, 'b': 640}
{'a': 46, 'b': 641}
{'a': 46, 'b': 172}
{'a': 46, 'b': 841}
{'a': 46, 'b': 843}
{'a': 46, 'b': 887}
{'a': 46, 'b': 970}
{'a': 46, 'b': 943}
{'a': 46, 'b': 14}
{'a': 46, 'b': 659}
{'a': 46, 'b': 174}
{'a': 46, 'b': 807}
{'a': 46, 'b': 872}
{'a': 134, 'b': 14}
{'a': 134, 'b': 525}
{'a': 134, 'b': 605}
{'a': 134, 'b': 608}
{'a': 134, 'b': 620}
{'a': 134, 'b': 640}
{'a': 134, 'b': 641}
{'a': 134, 'b': 642}
{'a': 134, 'b': 970}
{'a': 134, 'b': 23}
{'a': 134, 'b': 920}
{'a': 134, 'b': 921}
{'a': 134, 'b': 922}
{'a':

In [216]:
# Cargo la tabla motifs

# Leo el archivo donde estan los motifs
motifs_file=open("3_mfinderMycoh37rv_MEMBERS.txt","r")

# Inicio el cursor
cursor = cnx.cursor()

try:
    for line in motifs_file:      
        if str(line[0:8]) == "subgraph":
            tipoDeMotivo= int(line.split(" ")[3].rstrip())
        if line[0].isdigit():
            motivos=line.split("\t")
            C=motivos[0]
            D=motivos[1]
            E=motivos[2]       
            add = ("INSERT INTO motifs (C, D, E, motifID) VALUES (%(a)s, %(b)s, %(c)s, %(d)s)")
            data = {"a": C, "b": D, "c": E, "d":tipoDeMotivo}
            #print(data)
            cursor.execute(add,data)       
            cnx.commit()
finally:  
    cursor.close()
    motifs_file.close()
print("OK")

OK
